<a href="https://colab.research.google.com/github/sathya142003/coursera-test/blob/main/vulnerable_code_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: hugging face login

from huggingface_hub import notebook_login
notebook_login()


In [2]:
!pip install -q transformers datasets peft accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.6 MB/s eta 0:00:00


# step 1 : Load Model with QLoRA configuration

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from transformers import DataCollatorForSeq2Seq
import torch


In [4]:
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model_name = "Salesforce/codet5p-220m"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # Use float16 if on GPU
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/446M [00:00<?, ?B/s]

# Step 2 LoRA Adapters


In [9]:
"""lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    target_modules=["q", "v"]  # may vary depending on model
)

# Move the model to GPU before getting the PEFT model
if torch.cuda.is_available():
    model.to("cuda")


model = get_peft_model(model, lora_config)
model.print_trainable_parameters()"""
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],  # You can add 'k' and 'o' as well depending on the model architecture
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)

# ✅ Print trainable parameters (check if LoRA is attached)
model.print_trainable_parameters()


trainable params: 884,736 || all params: 223,766,784 || trainable%: 0.3954


The Devign dataset supports machine learning-based methods in identifying whether a given piece of code contains software vulnerabilities or not.


 Dataset Structure
Programming Language: C/C++

Source: Code snippets are extracted from real-world open-source projects in the Juliet Test Suite and other repositories.

Labeling:

0 → Non-vulnerable

1 → Vulnerable

Format: JSON, each sample includes:

Raw C code (function)

Label

#step 3: Load and Preprocess Devign Dataset

In [6]:
!pip install datasets git+https://github.com/huggingface/datasets.git

# Step 2: Clone the Devign dataset from Hugging Face Hub
!git clone https://huggingface.co/datasets/DetectVul/devign

# Step 3: Load the dataset locally
from datasets import load_dataset
import random

dataset = load_dataset("./devign")

#dataset = load_dataset("microsoft/Devign")

def preprocess(example):
    prompt = (
        "Classify the following function as 'vulnerable' or 'not vulnerable':\n\n"
        + example["func"]
    )
    label = "vulnerable" if example["target"] == 1 else "not vulnerable"

    input_enc = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    label_enc = tokenizer(label, truncation=True, padding="max_length", max_length=5)

    input_enc["labels"] = label_enc["input_ids"]
    return input_enc

tokenized = dataset.map(preprocess, remove_columns=dataset["train"].column_names)



  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-jht8dugo
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets.git /tmp/pip-req-build-jht8dugo
  Resolved https://github.com/huggingface/datasets.git to commit d9861d86be222884dabbd534a2db770c70c9b558
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
fatal: destination path 'devign' already exists and is not an empty directory.


Map:   0%|          | 0/21854 [00:00<?, ? examples/s]

Map:   0%|          | 0/2732 [00:00<?, ? examples/s]

Map:   0%|          | 0/2732 [00:00<?, ? examples/s]

# Step 4: Define Training Arguments and Trainer

In [52]:
training_args = Seq2SeqTrainingArguments(
    output_dir="codet5-qlora-devign-demo",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-4,
    num_train_epochs=10,
    eval_strategy="no",
    logging_steps=10,
    save_strategy="no",
    fp16=True,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"].select(range(128)),  # small subset demo
    eval_dataset=tokenized["validation"].select(range(64)),
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


/tmp/ipython-input-52-1869360129.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,0.273700
20,0.110600
30,0.215900
40,0.196000
50,0.150700
60,0.185800
70,0.165800
80,0.157400
90,0.196800
100,0.173900


KeyboardInterrupt: 

In [39]:
model.gradient_checkpointing_disable()


# Step 5:

In [89]:
def classify_code(code_snippet):
    prompt = (
        "Classify the following function as 'vulnerable' or 'not vulnerable':\n\n"
        + code_snippet
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=5)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test it
demo_code = "int f(char *s) { char buf[10]; strcpy(buf) return 0; }"
code = "void foo() { int *x = malloc(10); if (x == NULL) return; free(x); }"
print("Prediction:", classify_code(code))


Prediction: not vulner


In [32]:
model.save_pretrained("codet5p-devign-qlora")
tokenizer.save_pretrained("codet5p-devign-qlora")


('codet5p-devign-qlora/tokenizer_config.json',
 'codet5p-devign-qlora/special_tokens_map.json',
 'codet5p-devign-qlora/vocab.json',
 'codet5p-devign-qlora/merges.txt',
 'codet5p-devign-qlora/added_tokens.json',
 'codet5p-devign-qlora/tokenizer.json')